# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,17.73,81,100,2.17,ZA,1672971628
1,1,albany,42.6001,-73.9662,6.69,90,97,0.89,US,1672971620
2,2,santa barbara,34.7333,-120.0343,9.90,91,100,2.06,US,1672971565
3,3,cayenne,4.9333,-52.3333,23.02,100,75,1.03,GF,1672971353
4,4,new norfolk,-42.7826,147.0587,20.40,36,0,2.16,AU,1672971633


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

ideal_weather=city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21)&
                               (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
ideal_weather_df=ideal_weather.dropna(axis=0 , how="any")

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,mar del plata,-38.0023,-57.5575,21.72,93,0,2.57,AR,1672971607
40,40,cabo san lucas,22.8909,-109.9124,23.61,70,0,3.60,MX,1672971408
54,54,coahuayana,18.7333,-103.6833,22.66,92,0,0.30,MX,1672971710
127,127,champerico,14.3000,-91.9167,25.65,77,0,2.24,GT,1672971825
164,164,esperance,-33.8667,121.9000,24.99,45,0,3.77,AU,1672971882


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df[["City", "Country","Lat","Lng" , "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/Users/asalvalisoltani/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Lng,Humidity,Hotel Name
25,mar del plata,AR,-38.0023,-57.5575,93,
40,cabo san lucas,MX,22.8909,-109.9124,70,
54,coahuayana,MX,18.7333,-103.6833,92,
127,champerico,GT,14.3000,-91.9167,77,
164,esperance,AU,-33.8667,121.9000,45,
179,pisco,PE,-13.7000,-76.2167,83,
293,agua verde,MX,22.9000,-106.0500,64,
297,dhidhdhoo,MV,6.8833,73.1000,67,
345,lima,PE,-12.0432,-77.0282,71,
421,abu dhabi,AE,24.4667,54.3667,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel',
          "limit": 1,
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


/Users/asalvalisoltani/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


mar del plata - nearest hotel: Nuevo Ostende
cabo san lucas - nearest hotel: Comfort Rooms
coahuayana - nearest hotel: No hotel found
champerico - nearest hotel: Hotel y Restaurante El Submarino
esperance - nearest hotel: Hospitality
pisco - nearest hotel: La Portada
agua verde - nearest hotel: No hotel found
dhidhdhoo - nearest hotel: Cinnamon Island
lima - nearest hotel: Hotel Peach Garden
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
pozo colorado - nearest hotel: No hotel found
benjamin aceval - nearest hotel: No hotel found
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho


,City,Country,Lat,Lng,Humidity,Hotel Name
25,mar del plata,AR,-38.0023,-57.5575,93,Nuevo Ostende
40,cabo san lucas,MX,22.8909,-109.9124,70,Comfort Rooms
54,coahuayana,MX,18.7333,-103.6833,92,No hotel found
127,champerico,GT,14.3000,-91.9167,77,Hotel y Restaurante El Submarino
164,esperance,AU,-33.8667,121.9000,45,Hospitality
179,pisco,PE,-13.7000,-76.2167,83,La Portada
293,agua verde,MX,22.9000,-106.0500,64,No hotel found
297,dhidhdhoo,MV,6.8833,73.1000,67,Cinnamon Island
345,lima,PE,-12.0432,-77.0282,71,Hotel Peach Garden
421,abu dhabi,AE,24.4667,54.3667,71,Marriott Downtown Abu-Dhabi


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=0.5,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)